In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
login_url = 'https://stathead.com/users/login.cgi'
target_url = 'https://stathead.com/fbref/player-season-finder.cgi'
username = 'AndrewRH82'
password = 'LoboBear821$!@$'

In [3]:
session = requests.Session()

# Get the login page to retrieve any necessary hidden form fields
login_page = session.get(login_url)
soup = BeautifulSoup(login_page.content, 'html.parser')

# Find any hidden form fields (if present)
hidden_inputs = soup.find_all("input", type="hidden")
form_data = {input.get("name"): input.get("value") for input in hidden_inputs}

# Add your username and password to the form data
form_data['username'] = username
form_data['password'] = password

# Submit the login form
response = session.post(login_url, data=form_data)

# Check if login was successful
if "login" not in response.url:
    print("Login successful.")

    # Access the search page after logging in
    search_page = session.get(search_url)
    soup = BeautifulSoup(search_page.content, 'html.parser')

    # Locate the correct form by its ID
    search_form = soup.find("form", {"id": "player-season-finder"})
    if not search_form:
        print("Could not find the search form. Check the form ID.")
    else:
        print("Search form found.")

    # Extract the form action URL
    form_action = search_form.get('action', '')
    if form_action.startswith('/'):
        filter_url = f"https://stathead.com{form_action}"
    else:
        filter_url = f"{search_url}{form_action}"

    print(f"Submitting filters to: {filter_url}")

    # Define the search filters (update with actual filter names)
    filter_data = {
        'year_min': '2018-2019',  # Example: Filter for a specific season
        'year_max': '2018-2019',
        # Add other filters here if needed
    }

    # Include other hidden fields from the search form if required
    hidden_inputs = search_form.find_all("input", type="hidden")
    for input in hidden_inputs:
        filter_data[input.get("name")] = input.get("value")

    # Submit the filter form using POST
    filtered_response = session.post(filter_url, data=filter_data)

    # Check if the response is successful
    if filtered_response.status_code == 200:
        print("Filter form submitted successfully.")
    else:
        print(f"Filter form submission failed with status code: {filtered_response.status_code}")
        print("Trying GET request instead...")
        filtered_response = session.get(filter_url, params=filter_data)

    # Parse the search results page
    filtered_soup = BeautifulSoup(filtered_response.content, 'html.parser')

    # Extract and display data from the results table
    rows = filtered_soup.find_all('tr')
    if not rows:
        print("No data found. Check filter field names.")
    else:
        print("Data retrieved successfully.")
        for row in rows:
            print(row.get_text())

else:
    print("Login failed. Check your credentials.")

# Close the session
session.close()

Login successful.


NameError: name 'search_url' is not defined